In [ ]:
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class Network(nn.Module):
    def __init__(self,num_input=2,layers=[16, 32, 32, 16],num_output=2):
        super(Network,self).__init__()
        self.input_layer=nn.Linear(num_input,layers[0])
        self.hidden_layer=nn.ModuleList()
        for i in range(len(layers)-1):
            self.hidden_layer.append(nn.Linear(layers[i],layers[i+1]))
        self.output_layer=nn.Linear(layers[-1],num_output)
    def forward(self,out):
        out=torch.tanh(self.input_layer(out))
        for layer in self.hidden_layer:
            out=torch.tanh(layer(out))
        out=self.output_layer(out)
        return out

In [1]:
class Pinns:
    def __init__(self):
        #trnasfer to GPU if it is possible
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.network=Network().to(device)
        #Assumptions
        self.u0 = 1
        self.rho = 1
        self.nu = 0.01
       #Domain_Definition
        dx=0.01
        dy=0.01
        self.x=torch.arange(0,1+dx,dx)
        self.y=torch.arange(0,1+dy,dy)
        self.X=torch.stack(torch.meshgrid(self.x,self.y, indexing='xy')).reshape(2,-1).T
        #Boundary_condition_input
        dx_b=0.001
        dy_b=0.001
        self.x_b=torch.arange(0,1+dx_b,dx_b)
        self.y_b=torch.arange(0,1+dy_b,dy_b)
        self.rw = torch.stack(torch.meshgrid(self.x_b[-1],self.y_b)).reshape(2,-1).T
        self.lw = torch.stack(torch.meshgrid(self.x_b[0],self.y_b)).reshape(2,-1).T
        self.uw = torch.stack(torch.meshgrid(self.x_b,self.y_b[-1])).reshape(2,-1).T
        self.dw = torch.stack(torch.meshgrid(self.x_b,self.y_b[0])).reshape(2,-1).T
        self.uv_rw = torch.stack(torch.meshgrid(self.x_b[0],torch.zeros_like(self.y_b))).reshape(2,-1).T
        self.uv_lw = torch.stack(torch.meshgrid(self.x_b[0],torch.zeros_like(self.y_b))).reshape(2,-1).T
        self.uv_uw = torch.stack(torch.meshgrid(self.u0*self.x_b[-1],torch.zeros_like(self.y_b))).reshape(2,-1).T
        self.uv_dw = torch.stack(torch.meshgrid(self.x_b[0],torch.zeros_like(self.y_b))).reshape(2,-1).T
        #trnasfer tensor to GPU
        self.X=self.X.to(device)
        self.X.requires_grad = True
        self.rw=self.rw.to(device)
        self.rw.requires_grad = True
        self.lw=self.lw.to(device)
        self.lw.requires_grad = True
        self.uw=self.uw.to(device)
        self.uw.requires_grad = True
        self.dw=self.dw.to(device)
        self.dw.requires_grad = True
        self.uv_rw=self.uv_rw.to(device)
        self.uv_lw=self.uv_lw.to(device)
        self.uv_uw=self.uv_uw.to(device)
        self.uv_dw=self.uv_dw.to(device)
        # optimizer setting
        self.adam =  torch.optim.Adam(self.network.parameters())
        #Limited-memory Broyden-Fletcher-Goldfarb-Shanno (L-BFGS)
        self.optimizer = torch.optim.LBFGS(
            self.network.parameters(),
            lr=1.0,
            max_iter = 500,
            max_eval = 500,
            history_size = 50,
            tolerance_grad = 1e-7,
            tolerance_change = 1.0* np.finfo(float).eps,
            line_search_fn ="strong_wolfe"
        )
        #Error criterion Definition
        self.criterion=nn.MSELoss()
    def gradient(self,parameter1,parameter2,index):
        dP_dX = torch.autograd.grad(
            parameter1,
            parameter2,
            grad_outputs = torch.ones_like(parameter1),
            create_graph = True,
            retain_graph = True
        )[0]
        return dP_dX[:,index]
    def loss_f(self):
        #Restart Optimizer
        self.adam.zero_grad()
        self.optimizer.zero_grad()
        #output of NN
        self.spi_p_lw=self.network(self.lw)
        self.spi_lw=self.spi_p_lw[:,0]
        self.spi_p_rw=self.network(self.rw)
        self.spi_rw=self.spi_p_rw[:,0]
        self.spi_p_uw=self.network(self.uw)
        self.spi_uw=self.spi_p_uw[:,0]
        self.spi_p_dw=self.network(self.dw)
        self.spi_dw=self.spi_p_dw[:,0]
        self.spi_p_domain=self.network(self.X)
        self.spi_domain=self.spi_p_domain[:,0]
        self.p_domain=self.spi_p_domain[:,1]
        #compute u and v in boundary
        self.U_lw = self.gradient(self.spi_lw,self.lw,1)
        self.V_lw = -self.gradient(self.spi_lw,self.lw,0)
        self.U_rw = self.gradient(self.spi_rw,self.rw,1)
        self.V_rw = -self.gradient(self.spi_rw,self.rw,0)
        self.U_uw = self.gradient(self.spi_uw,self.uw,1)
        self.V_uw = -self.gradient(self.spi_uw,self.uw,0)
        self.U_dw = self.gradient(self.spi_dw,self.dw,1)
        self.V_dw = -self.gradient(self.spi_dw,self.dw,0)
        #loss data definition
        self.loss_data=self.criterion(self.U_lw,self.uv_lw[:,0])+\
                       self.criterion(self.V_lw,self.uv_lw[:,1])+\
                       self.criterion(self.U_rw,self.uv_rw[:,0])+\
                       self.criterion(self.V_rw,self.uv_rw[:,1])+\
                       self.criterion(self.U_uw,self.uv_uw[:,0])+\
                       self.criterion(self.V_uw,self.uv_uw[:,1])+\
                       self.criterion(self.U_dw,self.uv_dw[:,0])+\
                       self.criterion(self.V_dw,self.uv_dw[:,1])
        #compute u and v in domain
        self.u = self.gradient(self.spi_domain,self.X,1)
        self.v = -self.gradient(self.spi_domain,self.X,0)
        #compute derivations of u and v
        self.du_dx = self.gradient(self.u,self.X,0)
        self.du_dy = self.gradient(self.u,self.X,1)
        self.dv_dx = self.gradient(self.v,self.X,0)
        self.dv_dy = self.gradient(self.v,self.X,1)
        self.du_dxx = self.gradient(self.du_dx,self.X,0)
        self.du_dyy = self.gradient(self.du_dy,self.X,1)
        self.dv_dxx = self.gradient(self.dv_dx,self.X,0)
        self.dv_dyy = self.gradient(self.dv_dy,self.X,1)
        self.dp_dx = self.gradient(self.p_domain,self.X,0)
        self.dp_dy = self.gradient(self.p_domain,self.X,1)
        # compute equation loss
        u_eqn = self.u*self.du_dx + self.v*self.du_dy + self.dp_dx/self.rho - self.nu*(self.du_dxx + self.du_dyy)
        v_eqn = self.u*self.dv_dx + self.v*self.dv_dy + self.dp_dy/self.rho - self.nu*(self.dv_dxx + self.dv_dyy)
        #loss PDE definition
        self.loss_pde=self.criterion(u_eqn,torch.zeros_like(u_eqn))+self.criterion(v_eqn,torch.zeros_like(v_eqn))
        #loss function definition
        lambda_bc = 1.0
        lambda_pde = 1.0
        self.loss = lambda_bc * self.loss_data + lambda_pde * self.loss_pde
        self.loss.backward()
        return self.loss
    def train(self, num_epochs=1):
        self.network.train()
        for i in range(num_epochs):
            loss = self.loss_f()
            if i % 100 == 0:
                self.plot()
            if i % 10 == 0:
                print(f"Iteration {i}, Loss: {self.loss.item():.6f}")
            self.adam.step(self.loss_f)
        #self.optimizer.step(self.loss_f)
    def plot(self):
        import os
        os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
        self.network.eval()
        with torch.no_grad():
            u = self.u.reshape(len(self.x), len(self.y)).cpu().numpy().T
            v = self.v.reshape(len(self.x), len(self.y)).cpu().numpy().T
            psi = self.network(self.X)[:,0].reshape(len(self.x), len(self.y)).cpu().numpy().T
            p = self.network(self.X)[:,1].reshape(len(self.x), len(self.y)).cpu().numpy().T
        plt.figure(figsize=(10, 8))
        plt.subplot(2, 2, 1)
        contour1 = plt.contourf(self.x.cpu().numpy(),self.y.cpu().numpy(), psi, levels=50, cmap="jet")
        plt.colorbar(contour1)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Stream Function (Psi)")
        plt.subplot(2, 2, 2)
        contour2 = plt.contourf(self.x.cpu().numpy(), self.y.cpu().numpy(), u, levels=50, cmap="jet")
        plt.colorbar(contour2)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Velocity Component U")
        plt.subplot(2, 2, 3)
        contour3 = plt.contourf(self.x.cpu().numpy(), self.y.cpu().numpy(), v, levels=50, cmap="jet")
        plt.colorbar(contour3)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Velocity Component V")
        plt.subplot(2, 2, 4)
        contour4 = plt.contourf(self.x.cpu().numpy(), self.y.cpu().numpy(), p, levels=50, cmap="jet")
        plt.colorbar(contour4)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Pressure P")

        plt.tight_layout()
        plt.show()

In [ ]:
net = Pinns()
net.train(num_epochs=10000)
net.plot()